In [5]:
#!pip install transformers
#!pip install gensim
#!pip install nltk
#!pip install tensorflow --user

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import BertTokenizer, TFBertModel, BertForSequenceClassification, AdamW
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from torch.utils.data import DataLoader, TensorDataset
import string
import nltk
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split


nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
def message_cleaning(message):
    Test_punc_removed = [char for char in message if char not in string.punctuation]
    Test_punc_removed_join = ''.join(Test_punc_removed)
    Test_punc_removed_join_clean = [word for word in Test_punc_removed_join.split() if word.lower() not in stopwords.words('english')]
    return Test_punc_removed_join_clean

url = 'https://drive.google.com/uc?id={}'.format('1wHdWpDysG4hipEMnqJjGoQ1Oqf9mlzZF')
df = pd.read_csv(url)#,nrows=20)

tfidf_vectorizer = TfidfVectorizer(max_features=5000,analyzer = message_cleaning)  # You can adjust the number of features as needed
X_tfidf = tfidf_vectorizer.fit_transform(df['Review'])

y = df[['Rating']]
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

# Create and train a linear regression model
model1 = LinearRegression()
model1.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model1.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("MSE:",mse)
print("R2:",r2)

In [ ]:
new_review = "nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,  "
new_review_tfidf = tfidf_vectorizer.transform([new_review])
predicted_rating = model1.predict(new_review_tfidf)
print(predicted_rating)

import pickle
filename = 'finalized_model1.pkl'
pickle.dump(model1, open(filename, 'wb'))

y_pred = model1.predict(X_test)
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(mse)

In [ ]:
# read csv
url = 'https://drive.google.com/uc?id={}'.format('1wHdWpDysG4hipEMnqJjGoQ1Oqf9mlzZF')
df = pd.read_csv(url)

max_words = 5000  # You can adjust this value as needed
max_len = 100      # You can adjust this value as needed

# Tokenize the text data
tokenizer2 = Tokenizer(num_words=max_words)
tokenizer2.fit_on_texts(df['Review'])
X_seq = tokenizer2.texts_to_sequences(df['Review'])
X_padded = pad_sequences(X_seq, maxlen=max_len)

y = df[['Rating']]

X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)

# Create a model with an embedding layer and a Dense layer
model2 = tf.keras.Sequential()
model2.add(tf.keras.layers.Embedding(input_dim=max_words, output_dim=64, input_length=max_len))
model2.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model2.add(tf.keras.layers.Dense(1))

# Compile the model
model2.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model2.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

# Make predictions on the test set
y_pred = model2.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2): {r2}")



In [ ]:
# Define a new review text
new_review = "nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,  "

# Tokenize the new review
new_review_seq = tokenizer2.texts_to_sequences([new_review])
new_review_padded = pad_sequences(new_review_seq, maxlen=max_len)

# Make a prediction using the trained model
predicted_rating = model2.predict(new_review_padded)

# The predicted_rating is a NumPy array, so you can extract the predicted value
predicted_rating = predicted_rating[0][0]

# Print the predicted rating
print(f"Predicted Rating: {predicted_rating}")

model2.save('finalized_model2.h5')

In [ ]:
url = 'https://drive.google.com/uc?id={}'.format('1wHdWpDysG4hipEMnqJjGoQ1Oqf9mlzZF')
df = pd.read_csv(url,nrows=2000)

X = df['Review'].tolist()
y = df['Rating'].tolist()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)  # Regression with one output neuron
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(X_val, truncation=True, padding=True, return_tensors="pt")

# Create datasets and data loaders
train_dataset = TensorDataset(train_encodings["input_ids"], train_encodings["attention_mask"], torch.tensor(y_train, dtype=torch.float32))
val_dataset = TensorDataset(val_encodings["input_ids"], val_encodings["attention_mask"], torch.tensor(y_val, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Set up the optimizer and loss function for regression
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.MSELoss()

# Training loop
num_epochs = 5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = outputs.logits.squeeze(1)  # Squeeze to remove extra dimension
        loss = loss_fn(predictions, labels)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            predictions = outputs.logits.squeeze(1)
            val_loss = loss_fn(predictions, labels)
            val_losses.append(val_loss.item())
    
    mean_val_loss = np.mean(val_losses)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {mean_val_loss:.4f}")


# Perform any necessary post-processing or analysis on test_predictions


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/5, Validation Loss: 8.9199


In [ ]:
# Evaluate on test data if available and save the model
test_encodings = tokenizer(X_val, truncation=True, padding=True, return_tensors="pt")
test_dataset = TensorDataset(test_encodings["input_ids"], test_encodings["attention_mask"])
test_loader = DataLoader(test_dataset, batch_size=32)
test_predictions = []

model.eval()
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask = batch
        input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = outputs.logits.squeeze(1)
        test_predictions.extend(predictions.cpu().numpy())

# Save the trained model
torch.save(model.state_dict(), "bert_regression_model.pth")


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)  # Regression with one output neuron
model.load_state_dict(torch.load("bert_regression_model.pth"))  # Replace "bert_regression_model.pth" with the path to your trained model

model.eval()
input_text = "nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,  "
input_ids = tokenizer(input_text, truncation=True, padding=True, return_tensors="pt")["input_ids"]

# Make a prediction
with torch.no_grad():
    prediction = model(input_ids).logits.item()

print(f"Predicted Rating: {prediction:.0f}")


In [ ]:
#!pip install gensim
#!pip install nltk

In [7]:
import gensim
from gensim import corpora
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
url = 'https://drive.google.com/uc?id={}'.format('1wHdWpDysG4hipEMnqJjGoQ1Oqf9mlzZF')
df = pd.read_csv(url)
reviews = df['Review'].values.tolist()

# Tokenize and preprocess the reviews
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words and token not in punctuation]
    return tokens

tokenized_reviews = [preprocess(review) for review in reviews]

analyzer = SentimentIntensityAnalyzer()
def preprocessSentiment(words):
    positive_score = 0
    negative_score = 0
    for word in words:
        sentiment = analyzer.polarity_scores(word)
        positive_score += sentiment['pos']
        negative_score += sentiment['neg']

    if positive_score > negative_score:
        sentiment = "Good"
    elif positive_score < negative_score:
        sentiment = "Bad"
    else:
        sentiment = "Neutral"
    
    return sentiment

# Create a single dictionary for all reviews
dictionary = corpora.Dictionary(tokenized_reviews)

# Build an LDA model for each review
num_topics = 1  # You can adjust this number based on your requirements

# Print the topics and their top words for each review
for i, tokens in enumerate(tokenized_reviews):
    corpus = [dictionary.doc2bow(tokens)]
    lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

    print(f"Review {i + 1} Topics:")
    topics = lda_model.print_topics(num_words=5)
    for topic in topics:
        print(topic[1])
        text = topic[1]
        words = re.findall(r'"(.*?)"', text)
        print(words)
        sentiment=preprocessSentiment(words)
        print("sentiment:", sentiment)
        print()
        